## Train

In [1]:
import torch
torch.cuda.empty_cache()

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
import os

from trainer import Trainer, TrainerArgs

from TTS.config.shared_configs import BaseDatasetConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.layers.xtts.trainer.gpt_trainer import GPTArgs, GPTTrainer, GPTTrainerConfig, XttsAudioConfig
from TTS.utils.manage import ModelManager
from TTS.config import load_config

2024-02-28 10:12:32.239510: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
OUT_PATH = os.path.dirname(os.path.abspath("__file__"))

config_dataset = BaseDatasetConfig(
    formatter="ljspeech",
    meta_file_train="Mixed_formatted.txt",
    # meta_file_train = "No_Shouting_formatted.txt",
    path=os.path.join(OUT_PATH, "data/"),
    language = "en"
)
# Define here the dataset that you want to use for the fine-tuning on.
# config_dataset = BaseDatasetConfig(
#     formatter="ljspeech",
#     dataset_name="ljspeech",
#     path="/raid/datasets/LJSpeech-1.1_24khz/",
#     meta_file_train="/raid/datasets/LJSpeech-1.1_24khz/metadata.csv",
#     language="en",
# )

# Add here the configs of the datasets
DATASETS_CONFIG_LIST = [config_dataset]

In [4]:
# define audio config
audio_config = XttsAudioConfig(sample_rate=22050, dvae_sample_rate=22050, output_sample_rate=22050)
# training parameters config

In [5]:
# Define the path where XTTS v1.1.1 files will be downloaded
CHECKPOINTS_OUT_PATH = os.path.join(OUT_PATH, "xttsv2_checkpoint", "XTTS_v2.0_original_model_files/")
os.makedirs(CHECKPOINTS_OUT_PATH, exist_ok=True)


# DVAE files
DVAE_CHECKPOINT_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/dvae.pth"
MEL_NORM_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/mel_stats.pth"

# Set the path to the downloaded files
DVAE_CHECKPOINT = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(DVAE_CHECKPOINT_LINK))
MEL_NORM_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(MEL_NORM_LINK))

# download DVAE files if needed
if not os.path.isfile(DVAE_CHECKPOINT) or not os.path.isfile(MEL_NORM_FILE):
    print(" > Downloading DVAE files!")
    ModelManager._download_model_files([MEL_NORM_LINK, DVAE_CHECKPOINT_LINK], CHECKPOINTS_OUT_PATH, progress_bar=True)

# Download XTTS v2.0 checkpoint if needed
TOKENIZER_FILE_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/vocab.json"
XTTS_CHECKPOINT_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/model.pth"

# XTTS transfer learning parameters: You we need to provide the paths of XTTS model checkpoint that you want to do the fine tuning.
TOKENIZER_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(TOKENIZER_FILE_LINK))  # vocab.json file
XTTS_CHECKPOINT = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(XTTS_CHECKPOINT_LINK))  # model.pth file

# download XTTS v2.0 files if needed
if not os.path.isfile(TOKENIZER_FILE) or not os.path.isfile(XTTS_CHECKPOINT):
    print(" > Downloading XTTS v2.0 files!")
    ModelManager._download_model_files(
        [TOKENIZER_FILE_LINK, XTTS_CHECKPOINT_LINK], CHECKPOINTS_OUT_PATH, progress_bar=True
    )

In [6]:
# init args and config

XTTS_CHECKPOINT = "xttsv2_checkpoint/XTTS_v2.0_original_model_files/model.pth"

model_args = GPTArgs(
    max_conditioning_length=int(132300*2.5),  # 18 secs
    min_conditioning_length=66150,  # 3 secs
    debug_loading_failures=True,
    max_wav_length=255995*3,  # ~33 seconds
    max_text_length=700,
    mel_norm_file=MEL_NORM_FILE,
    dvae_checkpoint=DVAE_CHECKPOINT,
    xtts_checkpoint=XTTS_CHECKPOINT,  # checkpoint path of the model that you want to fine-tune
    tokenizer_file=TOKENIZER_FILE,
    gpt_num_audio_tokens=1026,
    gpt_start_audio_token=1024,
    gpt_stop_audio_token=1025,
    gpt_use_masking_gt_prompt_approach=True,
    gpt_use_perceiver_resampler=True,
    # gpt_max_text_tokens = 500,
    # gpt_max_prompt_tokens = 100
)

In [7]:
# Training sentences generations
# SPEAKER_REFERENCE = [
#     "data/wavs/1. Tyler1 THE_WORST_JUNGLER_EVER 1.wav"  # speaker reference to be used in training test sentences
# ]

SPEAKER_REFERENCE = ["data/wavs/" + wav for wav in os.listdir('data/wavs/') if "wav" in wav]

LANGUAGE = config_dataset.language

config = GPTTrainerConfig(
    output_path=OUT_PATH + '/xttsv2_checkpoint',
    model_args=model_args,
    run_name="tyler1_xttsv2",
    project_name="tyler1",
    dashboard_logger="tensorboard",
    # logger_uri=None,
    audio=audio_config,
    batch_size=1,
    # batch_group_size=48,
    eval_batch_size=1,
    num_loader_workers=2,
    # eval_split_max_size=256,
    print_step=50,
    plot_step=100,
    log_model_step=1000,
    save_step=10000,
    save_n_checkpoints=1,
    save_checkpoints=True,
    # target_loss="loss",
    print_eval=True,
    # Optimizer values like tortoise, pytorch implementation with modifications to not apply WD to non-weight parameters.
    optimizer="AdamW",
    optimizer_wd_only_on_weights=True, # for multi-gpu training please make it False
    optimizer_params={"betas": [0.9, 0.96], "eps": 1e-8, "weight_decay": 1e-2},
    lr=5e-06,  # learning rate
    lr_scheduler="MultiStepLR",
    # it was adjusted accordly for the new step scheme
    lr_scheduler_params={"milestones": [50000 * 18, 150000 * 18, 300000 * 18], "gamma": 0.5, "last_epoch": -1},
    test_sentences=[
        {
            "text": "So he starts off level one just doing this. Like oh he's gonna like bro he losing, doesn't hit a single thing. Look at it like- what the fuck. Like bro okay whatever. Watch this top dive bro. I solo made Vayne one HP, right? Just wait out and fucking ghost, you twat.",
            "speaker_wav": SPEAKER_REFERENCE,
            "language": LANGUAGE,
        },
        {
            "text": "Okay whatever you're auto-ing a ward sure it's fine. Bro what are you d- just wait you fucking freak. Where's he walking to by the way? What the fuck! It's not a win-trade this guy played as our Jarvan too. He's a one-trick like bro.",
            "speaker_wav": SPEAKER_REFERENCE,
            "language": LANGUAGE,
        },
        {
            "text": "Hey! Sup? It's me, Tyler1 ready for the pre-alpha. We're back baby!",
            "speaker_wav": SPEAKER_REFERENCE,
            "language": LANGUAGE,
        },
    ],
    # mixed_precision = False
)

In [8]:
def formatter(root_path, manifest_file, **kwargs):  # pylint: disable=unused-argument
    """Assumes each line as ```<filename>|<transcription>```
    """
    txt_file = os.path.join(root_path, manifest_file)
    items = []
    speaker_name = "Tyler1"
    with open(txt_file, "r", encoding="utf-8") as ttf:
        for line in ttf:
            cols = line.split("|")
            wav_file = os.path.dirname(os.path.abspath('__file__')) + f"/data/wavs/{cols[0]}.wav"
            text = cols[1]
            # print(text)
            items.append({"text":text, "audio_file":wav_file, "speaker_name":speaker_name, "root_path": root_path})
    return items


train_samples, eval_samples = load_tts_samples(
    config_dataset,
    # DATASETS_CONFIG_LIST,
    eval_split=True,
    # eval_split_max_size=config.eval_split_max_size,
    eval_split_size=0.1,
    formatter = formatter
)

 | > Found 558 files in /home/sagemaker-user/dist/main/data


In [9]:
# init the model from config
model = GPTTrainer.init_from_config(config)

>> DVAE weights restored from: /home/sagemaker-user/dist/main/xttsv2_checkpoint/XTTS_v2.0_original_model_files/dvae.pth


In [10]:
# init the trainer and 🚀

# model_path = "xttsv2_checkpoint/tyler1_xttsv2-February-19-2024_05+30AM-2b31060/"
trainer = Trainer(
    TrainerArgs(
        # continue_path = model_path,  # xtts checkpoint is restored via xtts_checkpoint key so no need of restore it using Trainer restore_path parameter
        skip_train_epoch=False,
        start_with_eval=True,
        grad_accum_steps=256, # batch_size * grad_accum_steps >= 256,
    ),
    config,
    output_path=OUT_PATH,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
)

 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: False
 | > Precision: float32
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 48
 | > Num. of Torch Threads: 1
 | > Torch seed: 1
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=/home/sagemaker-user/dist/main/xttsv2_checkpoint/tyler1_xttsv2-February-28-2024_10+12AM-e526ca1

 > Model has 518442047 parameters


In [ ]:
trainer.fit() 


 > EPOCH: 0/1000
 --> /home/sagemaker-user/dist/main/xttsv2_checkpoint/tyler1_xttsv2-February-28-2024_10+12AM-e526ca1


 > Filtering invalid eval samples!!



 > EVALUATION 



 > Total eval samples after filtering: 55


   --> STEP: 0
     | > loss_text_ce: 0.0258428156375885  (0.0258428156375885)
     | > loss_mel_ce: 3.432394504547119  (3.432394504547119)
     | > loss: 3.458237409591675  (3.458237409591675)

   --> STEP: 1
     | > loss_text_ce: 0.019895030185580254  (0.019895030185580254)
     | > loss_mel_ce: 4.103507041931152  (4.103507041931152)
     | > loss: 4.123402118682861  (4.123402118682861)

   --> STEP: 2
     | > loss_text_ce: 0.023496203124523163  (0.021695616655051708)
     | > loss_mel_ce: 4.415037155151367  (4.25927209854126)
     | > loss: 4.438533306121826  (4.280967712402344)

   --> STEP: 3
     | > loss_text_ce: 0.03131064027547836  (0.02490062452852726)
     | > loss_mel_ce: 4.0959882736206055  (4.204844156901042)
     | > loss: 4.127298831939697  (4.229744752248128)

   --> STEP: 4
     | > loss_text_ce: 0.024248966947197914  (0.024737710133194923)
     | > loss_mel_ce: 4.163119316101074  (4.19441294670105)
     | > loss: 4.187368392944336  (4.21915066242218)

   --> STEP: 

 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.020982216905664514 (+0)
     | > avg_loss_text_ce: 0.025445595832058677 (+0)
     | > avg_loss_mel_ce: 4.1630218161476975 (+0)
     | > avg_loss: 4.188467409875658 (+0)


 > EPOCH: 1/1000
 --> /home/sagemaker-user/dist/main/xttsv2_checkpoint/tyler1_xttsv2-February-28-2024_10+12AM-e526ca1

 > TRAINING (2024-02-28 10:14:01) 


 > Sampling by language: dict_keys(['en'])



   --> TIME: 2024-02-28 10:14:02 -- STEP: 0/503 -- GLOBAL_STEP: 0
     | > loss_text_ce: 0.02602170594036579  (0.02602170594036579)
     | > loss_mel_ce: 3.3024559020996094  (3.3024559020996094)
     | > loss: 0.013001865707337856  (0.013001865707337856)
     | > current_lr: 5e-06 
     | > step_time: 0.3264  (0.32635974884033203)
     | > loader_time: 0.7926  (0.792572021484375)


   --> TIME: 2024-02-28 10:14:16 -- STEP: 50/503 -- GLOBAL_STEP: 50
     | > loss_text_ce: 0.029459068551659584  (0.026212509870529175)
     | > loss_mel_ce: 4.78672456741333  (nan)
     | > loss: 0.01881321705877781  (nan)
     | > current_lr: 5e-06 
     | > step_time: 0.2741  (0.2664296293258668)
     | > loader_time: 0.0104  (0.018505811691284187)


   --> TIME: 2024-02-28 10:14:30 -- STEP: 100/503 -- GLOBAL_STEP: 100
     | > loss_text_ce: 0.029689673334360123  (0.02601445695385337)
     | > loss_mel_ce: 6.378173828125  (nan)
     | > loss: 0.025030717253684998  (nan)
     | > current_lr: 5e-06 
     |

##  Generate

In [ ]:
from TTS.api import TTS
from subprocess import getoutput
from IPython.display import Audio

# using the default version set in 🐸TTS
# tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", gpu=True)

# using a specific version
# 👀 see the branch names for versions on https://huggingface.co/coqui/XTTS-v2/tree/main
# ❗some versions might be incompatible with the API
# tts = TTS("xtts_v2.0.2", gpu=True)

# getting the latest XTTS_v2
tts = TTS("xtts").to("cuda")

SPEAKER_REFERENCE = ["data/wavs/" + wav for wav in os.listdir('data/wavs/') if "wav" in wav]

tts.tts_to_file(text= "What's up? What's up?! It's Tyler1 baby! Back with the voice",
                file_path="generated_audio/xtts_output.wav",
                speaker_wav=SPEAKER_REFERENCE,
                language="en"
               )

Audio("generated_audio/xtts_output.wav")

In [1]:
import os
import torch
import torchaudio
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts
from subprocess import getoutput
from IPython.display import Audio
import time


# Add here the xtts_config path
CONFIG_PATH = "xttsv2_checkpoint/tyler1_xttsv2-February-28-2024_01+16PM-e526ca1/config.json"
# Add here the vocab file that you have used to train the model
TOKENIZER_PATH = "xttsv2_checkpoint/XTTS_v2.0_original_model_files/vocab.json"
# Add here the checkpoint that you want to do inference with
XTTS_CHECKPOINT = "xttsv2_checkpoint/tyler1_xttsv2-February-28-2024_01+16PM-e526ca1/best_model.pth"

# List of all wavs for speaker reference
wavs = getoutput("ls data/wavs/*.wav").split("\n")
# Add here the speaker reference
SPEAKER_REFERENCE = ["data/wavs/" + wav for wav in os.listdir('data/wavs/') if "wav" in wav]



start = time.time()
config = XttsConfig()
config.load_json(CONFIG_PATH)
model = Xtts.init_from_config(config)
model.load_checkpoint(config, checkpoint_path=XTTS_CHECKPOINT, vocab_path=TOKENIZER_PATH, use_deepspeed=False)
model.to("cuda")

gpt_cond_latent, speaker_embedding = model.get_conditioning_latents(audio_path= SPEAKER_REFERENCE)

end = time.time()
print(end-start)

2024-03-05 07:29:42.320106: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


41.310394048690796


In [2]:
test_sentences=["So he starts off level one just doing this. Like oh he's gonna like bro he losing, doesn't hit a single thing. Look at it like- what the fuck. Like bro okay whatever. Watch this top dive bro. I solo made Vayne one HP, right? Just wait out and fucking ghost, you twat.",
            "Okay whatever you're auto-ing a ward sure it's fine. Bro what are you d- just wait you fucking freak. Where's he walking to by the way? What the fuck! It's not a win-trade this guy played as our Jarvan too. He's a one-trick like bro.",
            "Hey! Sup? It's me, Tyler1 ready for the pre-alpha. We're back baby!",]

In [6]:
import time
sample = 0

start = time.time()
out = model.inference(
    test_sentences[sample],
    "en",
    gpt_cond_latent,
    speaker_embedding,
    temperature=0.2, # Add custom parameters here
    top_k = model.config.top_k,
    top_p = model.config.top_p,
    # length_penalty = model.config.length_penalty,
)

# output wav path
OUTPUT_WAV_PATH = "generated_audio/xtts_output{}.wav".format(sample)

torchaudio.save(OUTPUT_WAV_PATH, torch.tensor(out["wav"]).unsqueeze(0), 24000)

end = time.time()
print(end-start)

Audio(OUTPUT_WAV_PATH)

10.653507947921753


In [12]:
len(test_sentences[0])

267